In [ ]:
#|code-fold: true
#|code-summary: "Code Preamble"

import sympy
from sympy import symbols, diff, ln
from sympy.codegen.cfunctions import log2
import functools
import matplotlib.pyplot as plt

x0=symbols("x0")
x1=symbols("x1")

def call(f, x):
    return f(x)

def mapv(a, b):
    return list(map(a, map(lambda x: 1.0 * x, b)))

def mapv_indexed(f, coll):
    return list(map(lambda x: [x, f(x)], map(lambda x: 1.0 * x, coll)))

class PipeAsCall(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __call__(self, x):
        return self.func(x)

class Transducer(PipeAsCall):
  pass

def partial(*args):
    return PipeAsCall(
        functools.partial(*args))

def D(f):
  var = symbols("var")
  return PipeAsCall(sympy.lambdify([var], diff(f(var), var), "sympy"))

def rnd(expr):
     return expr.xreplace({n: round(n, 3) for n in expr.atoms(sympy.Number)})

def plot(f, x):
    if (type(f)==str) or (type(f)==sympy.core.numbers.Float):
       return "That's not something you can plot"
    else:
       plt.plot(x, mapv(f, x), "o")

def plotting(f):
    if type(f) == Transducer:
        return partial(lambda f, coll: plt.plot(list(map(lambda x: x[0],f(coll))),list(map(lambda x: x[1],f(coll))), "o"), f)
    else:
        return partial(lambda f, x: plt.plot(x, mapv(f, x), "o"), f)

isPositive = lambda n: True if n>0 else False

def smaller(x):
    return lambda n: True if n<x else False

def filtering(f):
    return Transducer(functools.partial(lambda f,coll: list(filter(f, coll)), f))

def mapping(f):
    return Transducer(functools.partial(mapv_indexed, f))

def compose(f, g):
    return Transducer(lambda x: f(g(x)))

In [ ]:
frequencies = [16.35, 17.32, 18.35, 19.45, 20.6, 21.83, 23.12, 24.5, 25.96, 27.5, 29.14, 30.87, 32.7, 34.65, 36.71, 38.89, 41.2, 43.65, 46.25, 49.0, 51.91, 55.0, 58.27, 61.74, 65.41, 69.3, 73.42, 77.78, 82.41, 87.31, 92.5, 98.0, 103.83, 110.0, 116.54, 123.47, 130.81, 138.59, 146.83, 155.56, 164.81, 174.61, 185.0, 196.0, 207.65, 220.0, 233.08, 246.94, 261.63, 277.18, 293.66, 311.13, 329.63, 349.23, 369.99, 392.0, 415.3, 440.0, 466.16, 493.88, 523.25, 554.37, 587.33, 622.25, 659.25, 698.46, 739.99, 783.99, 830.61, 880.0, 932.33, 987.77, 1046.5, 1108.73, 1174.66, 1244.51, 1318.51, 1396.91, 1479.98, 1567.98, 1661.22, 1760.0, 1864.66, 1975.53, 2093.0, 2217.46, 2349.32, 2489.0, 2637.0, 2793.83, 2959.96, 3135.96, 3322.44, 3520.0, 3729.31, 3951.0, 4186.0, 4434.92, 4698.63, 4978.0, 5274.0, 5587.65, 5919.91, 6271.93, 6644.88, 7040.0, 7458.62, 7902.13]

In [ ]:
plot(log2, [1, 2, 4, 8])

In [ ]:
plot("log2(x / 220) * 7", [1, 2, 4, 8])

In [ ]:
def Poe(intersect, steps, x):
    return log2(x/intersect) * steps

Poe(220, 7, 440)

In [ ]:
def log2b(intersect, steps):
    return partial(Poe, intersect, steps)

In [ ]:
plot(Poe(220, 7, 440), [1, 2, 4, 8])

In [ ]:
plot(partial(Poe, 220, 7), [1, 2, 4, 8])

In [ ]:
plot(log2b(220, 7), [1, 2, 4, 8])

In [ ]:
call(log2b(220, 7), 8)

In [ ]:
call(log2b(220, 7), x0)

In [ ]:
call(log2b(1, 0.7), x1)

In [ ]:
call(D(log2b(1, 0.7)), x0).evalf()

In [ ]:
call(D(ln), x0)

In [ ]:
(D(ln) | x0)

In [ ]:
(plotting(ln) | frequencies)

In [ ]:
(plotting(ln) |
  (filtering(smaller(900)) |
    frequencies))

In [ ]:
(plotting
  (compose
    (mapping(ln), 
     filtering(smaller(900)))) |
  frequencies)

In [ ]:
(plotting
  (compose
    (mapping(D(ln)), 
     filtering(smaller(900)))) |
  frequencies)